**<u>Fun with Return and Continue!</u>**

The notebook will help document when to use return and continue effectively.

<span style="color: var(--vscode-foreground);">Often when an unexpected exception occurs in a function, we want to end the function early and rely on the "return" keyword.</span>

Some great articles:

[The PowerShell return keyword | mikefrobbins.com](https:\mikefrobbins.com\2015\07\23\the-powershell-return-keyword\)

[Flow control - PowerShell | Microsoft Learn](https:\learn.microsoft.com\en-us\powershell\scripting\learn\ps101\06-flow-control?view=powershell-7.4#break-continue-and-return)

In [3]:
function Get-Stuff1 {
    param(
        [string]$InputObject
    )
    Write-Host "Starting: $InputObject"
    if ($InputObject -eq 'z') {
        Write-Warning "We shouldn't continue"
        return
    }
    Write-Host "Rest of the code"
}
Get-Stuff1 -InputObject 'a'
Get-Stuff1 -InputObject 'z'


Starting: a


Rest of the code


Starting: z


This works great!  We can also use it in the same way for expected values to accomplish simple flow control, which can make complex case switching easier.

The "return" keyword is "designed to exit out of the existing scope" and differs from continue and break in that it ignores loops and is looking at the script block.

In [2]:
function Get-Stuff2 {
    Write-Host "Start of 'continue' example"
    $Values = @(1, 2, 3)
    foreach ($Value in $Values) {
        if ($Value -eq 2) {
            continue
        }
        Write-Host $Value
    }
    Write-Host "End of 'continue' example reached"
}
function Get-Stuff3 {
    Write-Host "Start of 'break' example"
    $Values = @(1, 2, 3)
    foreach ($Value in $Values) {
        if ($Value -eq 2) {
            break
        }
        Write-Host $Value
    }
    Write-Host "End of 'break' example reached"
}
function Get-Stuff4 {
    Write-Host "Start of 'return' example"
    $Values = @(1, 2, 3)
    foreach ($Value in $Values) {
        if ($Value -eq 2) {
            return
        }
        Write-Host $Value
    }
    Write-Host "End of 'return' example reached"
}
Get-Stuff2
Get-Stuff3
Get-Stuff4

Start of 'continue' example
1
3
End of 'continue' example reached
Start of 'break' example
1
End of 'break' example reached
Start of 'return' example
1


We can see that using continue skips only the "2" but still prints "3," whereas break exists the foreach entirely.  Return exits the entire function.  Do and while behave in the same way.  ForEach-Object behaves differently, which we will get to.

For now, let's look at an edge case for "return."  If we are using the begin/process/end blocks in an advanced function, "return" only exists the current block.

In [4]:
function Get-Stuff5 {
    [CmdletBinding()]
    param (
        [Parameter(ValueFromPipeline)][string]$InputObject,
        [switch]$StopEarly
    )
    begin {
        Write-Output "Started begin"
        if ($StopEarly) {
            Write-Warning "Stopping early due to failed condition"
            return
        }
        Write-Output "We're not stopping early"
    }
    process {
        Write-Output "Processing $InputObject"
    }
    end {
        Write-Output "Reached end"
    }
}
@(1, 2, 3) | Get-Stuff5 -StopEarly

Started begin
Processing 1
Processing 2
Processing 3
Reached end


Return only stopped reading the begin block, even though a bad condition was found in the begin block.  It did it's job of exiting the "current scope."  We can use continue _in this case_ to exit the entire function from the begin block.

In [7]:
function Get-Stuff6 {
    [CmdletBinding()]
    param (
        [Parameter(ValueFromPipeline)][string]$InputObject,
        [switch]$StopEarly
    )
    begin {
        Write-Output "Started begin"
        if ($StopEarly) {
            Write-Warning "Stopping early due to failed condition"
            continue
        }
        Write-Output "We're not stopping early"
    }
    process {
        Write-Output "Processing $InputObject"
        
    }
    end {
        Write-Output "Reached end"
    }
}
@(1, 2, 3) | Get-Stuff6 -StopEarly

Started begin


This works in the begin block, but we wouldn't want this behavior in the process block.  If an invalid condition is met in a single value for the input, the rest of the input should still be returned.

In [10]:
function Get-Stuff7 {
    [CmdletBinding()]
    param (
        [Parameter(ValueFromPipeline)][string]$InputObject
    )
    begin {
        Write-Output "Started begin"
    }
    process {
        if ($InputObject -eq 2) {
            Write-Warning "Stopping early due to failed condition"
            continue
        }
        Write-Output "Processing $InputObject"
    }
    end {
        Write-Output "Reached end"
    }
}
@(1, 2, 3) | Get-Stuff7

Started begin
Processing 1


The 1st input ran succesfully.  The 2nd failed, and exited, which is great.  But what happened to the third input?  It's rare that a problem in the middle of a collection should cause inputs after _but not before_ to also be skipped.  Maybe that's the behavior you want for certain applications, but often not.  So how do we handle this?  In this case we have to use "return" again.

In [11]:
function Get-Stuff8 {
    [CmdletBinding()]
    param (
        [Parameter(ValueFromPipeline)][string]$InputObject
    )
    begin {
        Write-Output "Started begin"
    }
    process {
        if ($InputObject -eq 2) {
            Write-Warning "Stopping early due to failed condition"
            return
        }
        Write-Output "Processing $InputObject"
    }
    end {
        Write-Output "Reached end"
    }
}
@(1, 2, 3) | Get-Stuff8

Started begin
Processing 1
Processing 3
Reached end


When writing advanced functions that accept pipeline input, I often will also allow array values to be entered as simple parameters, instead of _forcing_ use of the pipeline.  This is accomplished by using a foreach inside the process block, and in this case we would want to use a continue again!

In [25]:
function Get-Stuff9 {
    [CmdletBinding()]
    param (
        [Parameter(ValueFromPipeline)][int[]]$InputObject
    )
    begin {
        Write-Output "Started begin"
    }
    process {
        foreach ($item in $InputObject) {
            if ($item -eq 2) {
                Write-Warning "Stopping early due to failed condition : $item"
                return
            }
            Write-Output "Processing $item"
        }
    }
    end {
        Write-Output "Reached end"
    }
}
Write-Host "Using pipeline"
1..3 | Get-Stuff9
Write-Host "Using parameter with array"
Get-Stuff9 -InputObject @(1..3)

Using pipeline
Started begin
Processing 1
Processing 3
Reached end
Using parameter with array
Started begin
Processing 1
Reached end


Get-Stuff8 is why when stopping processing early for a single value in a ForEach-Object we should generally use return instead of continue.

In [12]:
@(1, 2, 3) | ForEach-Object {
    if ($_ -eq 2) {
        Write-Warning "Stopping early due to failed condition"
        return
    }
    Write-Output "Processing $InputObject"
}

Processing 
Processing 


Exiting function early from within foreach in begin:

In [15]:
function Get-Stuff10 {
    [CmdletBinding()]
    param ()
    begin {
        # Check some elements first before ever entering the process block.
        foreach ($i in 1..3) {
            if ($i -eq 2) {
                continue
            }
        }
        Write-Output "This is the begin block"
    }
    process {
        Write-Output "This is the process block"
    }
    end {
        Write-Output "This is the end block"
    }

}
Get-Stuff10

This is the begin block
This is the process block
This is the end block


We can't use continue in the begin block to exit the function when the continue is inside a loop.  What do we do?  Here's what I would do.  I'm not sure if there is a better way.

In [19]:
function Get-Stuff11 {
    [CmdletBinding()]
    param ()
    begin {
        # Check some elements first before ever entering the process block.
        $ExitEarly = $false
        foreach ($i in 1..3) {
            if ($i -eq 2) {
                $ExitEarly = $true
                continue
            }
        }
        if ($ExitEarly) {
            Write-Output "Exiting early"
            continue
        }
        Write-Output "This is the begin block"
    }
    process {
        Write-Output "This is the process block"
    }
    end {
        Write-Output "This is the end block"
    }
}
Get-Stuff11

Exiting early
